In [1]:
from game import Game
import time
import os
import re

## Load maps

In [2]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [3]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [4]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [5]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [6]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [7]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [8]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [9]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


## Solvers

I put a time limit for each solver:

In [10]:
TIME_LIMIT = 60

### BFS

In [11]:
# TODO: Must return moves (if there is no solution return None), number of visited states
from collections import deque

def solver_bfs(game_map, start_time):
    game = Game(game_map)
    queue = deque([(game.get_player_position(), tuple(game.get_box_locations()), "")])
    visited = set()
    
    while queue:
        if(time.time() - start_time > TIME_LIMIT):
            return None, -1
        player_position, boxes_position, path = queue.popleft()
        state_hash = (player_position, boxes_position)
        
        if state_hash in visited:
            continue
        visited.add(state_hash)


        for move in ['U', 'D', 'L', 'R']:
            game.set_player_position(player_position)
            game.set_box_positions(boxes_position)

            if game.apply_move(move):
                if game.is_game_won():
                    return path + move, len(visited)
                new_state = (game.get_player_position(), tuple(game.get_box_locations()), path + move)
                queue.append(new_state)
                
    
    return None, 0

### DFS

In [12]:
# TODO: Must return moves, number of visited states
def solver_dfs(game_map, start_time, depth_limit=1000):
    game = Game(game_map)
    stack = [(game.get_player_position(), tuple(game.get_box_locations()), "")]
    visited = set()

    while stack:
        if(time.time() - start_time > TIME_LIMIT):
            return None, -1
        
        player_position, boxes_position, path = stack.pop()
        state_hash = (player_position, boxes_position)

        if state_hash in visited:
            continue

        visited.add(state_hash)

        if len(path) >= depth_limit:
            continue

        for move in ['U', 'D', 'L', 'R']:
            game.set_player_position(player_position)
            game.set_box_positions(boxes_position)

            if game.is_game_won():
                    return path, len(visited)
            
            if game.apply_move(move):
                if game.is_game_won():
                    return path + move, len(visited)
                new_state = (game.get_player_position(), tuple(game.get_box_locations()), path + move)  
                stack.append(new_state)
    
    return None, 0


### IDS

In [13]:
def solver_ids(game_map, start_time, max_depth=1000):
    game = Game(game_map)
    initial_player_position = game.get_player_position()
    initial_boxes_position = tuple(game.get_box_locations())
    
    total_visited = 0
    
    for depth in range(max_depth):
        
        visited = set()
        queue = deque([(initial_player_position, initial_boxes_position, "")])
        
        while queue:
            if time.time() - start_time > TIME_LIMIT:
                return None, -1
                
            player_position, boxes_position, path = queue.popleft()
            
            if (player_position, boxes_position) in visited:
                continue
                
            visited.add((player_position, boxes_position))
            total_visited += 1

            game.set_player_position(player_position)
            game.set_box_positions(boxes_position)
            if game.is_game_won():
                return path, total_visited
                
            if len(path) >= depth:
                continue
                
            for move in ['U', 'D', 'L', 'R']:
                game.set_player_position(player_position)
                game.set_box_positions(boxes_position)
                
                if game.apply_move(move):
                    new_player_pos = game.get_player_position()
                    new_boxes_pos = tuple(game.get_box_locations())
                    new_path = path + move
                    
                    queue.append((new_player_pos, new_boxes_pos, new_path))
    
    return None, total_visited

### A*

#### Manhatann Distance and heuristic functions

 - *First heuristic*:
    Sum of Manhattan distances from each box to its goal. 
    This is admissible cause box can move at best one step per move.

In [14]:
def manhattan_distance(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def heuristic_1(game):
    
    boxes = game.get_box_locations()
    goals = game.get_goal_locations()
    
    total_distance = 0
    for i in range(len(boxes)):
        total_distance += manhattan_distance(boxes[i], goals[i])
        
    return total_distance

- *Second heuristic*:
    Sum of Manhattan distances and player distance to nearest box.

In [15]:
def heuristic_2(game):

    boxes = game.get_box_locations()
    goals = game.get_goal_locations()
    player_pos = game.get_player_position()
   
    total_distance = 0
    for i in range(len(boxes)):
        total_distance += manhattan_distance(boxes[i], goals[i])
    
    min_distance_to_box = float('inf')
    for box in boxes:
        if box in goals:  #
            continue
        dist = manhattan_distance(player_pos, box)
        if dist < min_distance_to_box:
            min_distance_to_box = dist
    
    if min_distance_to_box < float('inf'):
        total_distance += min_distance_to_box
        
    return total_distance

- *Third heuristic*:
    1. Sum of Manhattan distances from each box to its goal
    2. Player distance to the nearest box that's not on its goal
    3. Penalty for boxes in corners or against walls (deadlock detection)

In [16]:
def heuristic_3(game):

    boxes = game.get_box_locations()
    goals = game.get_goal_locations()
    player_pos = game.get_player_position()
    
    total_distance = 0
    boxes_not_on_goal = []
    
    for i in range(len(boxes)):
        box_pos = boxes[i]
        goal_pos = goals[i]
        box_to_goal = manhattan_distance(box_pos, goal_pos)
        total_distance += box_to_goal
        
        if box_pos != goal_pos:
            boxes_not_on_goal.append(box_pos)
    
    if boxes_not_on_goal:
        min_distance_to_box = min(manhattan_distance(player_pos, box) for box in boxes_not_on_goal)
        total_distance += min_distance_to_box 
    
    for box_pos in boxes:
        if box_pos in goals: 
            continue
            
        wall_count = 0
        for direction in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            adj_pos = (box_pos[0] + direction[0], box_pos[1] + direction[1])
            if game.is_wall(adj_pos):
                wall_count += 1
        
        if wall_count >= 1: 
            total_distance += 5  
    
    return total_distance

- *Fourth heuristic*: count of the boxes that are not in their goal cell

In [17]:
def heuristic_4(game):
    boxes = game.get_box_locations()
    goals = game.get_goal_locations()

    distance = 0
    for i in range(len(boxes)):
        if boxes[i] != goals[i]:
            distance += 1

    return distance        

In [18]:
import heapq

def solver_astar(game_map, start_time, heuristic=heuristic_3):
    game = Game(game_map)
    start_state = (game.get_player_position(), tuple(game.get_box_locations()), "")
    
    heap = [(0 + heuristic(game), 0, start_state)] 
    visited = set()

    while heap:
        if(time.time() - start_time > TIME_LIMIT):
            return None, -1
        _, g, (player_position, boxes_position, path) = heapq.heappop(heap)
        state_hash = (player_position, boxes_position)

        if state_hash in visited:
            continue
        visited.add(state_hash)

        game.set_player_position(player_position)
        game.set_box_positions(boxes_position)

        if game.is_game_won():
            return path, len(visited)

        for move in ['U', 'D', 'L', 'R']:
            if game.apply_move(move):
                new_state = (game.get_player_position(), tuple(game.get_box_locations()), path + move)
                new_g = g + 1
                new_f = new_g + heuristic(game)

                heapq.heappush(heap, (new_f, new_g, new_state))

                game.set_player_position(player_position)
                game.set_box_positions(boxes_position)

    return None, 0

### Weighted A*

In [19]:
def solver_weighted_astar(game_map, start_time, heuristic=heuristic_3, w=5):
    game = Game(game_map)
    start_state = (game.get_player_position(), tuple(game.get_box_locations()), "")

    heap = [(0 + w * heuristic(game), 0, start_state)] 
    visited = set()

    while heap:
        if(time.time() - start_time > TIME_LIMIT):
            return None, -1

        _, g, (player_position, boxes_position, path) = heapq.heappop(heap)
        state_hash = (player_position, boxes_position)

        if state_hash in visited:
            continue
        visited.add(state_hash)

        game.set_player_position(player_position)
        game.set_box_positions(boxes_position)

        if game.is_game_won():
            return path, len(visited)

        for move in ['U', 'D', 'L', 'R']:
            if game.apply_move(move):
                new_state = (game.get_player_position(), tuple(game.get_box_locations()), path + move)
                new_g = g + 1
                new_f = new_g + w * heuristic(game) 

                heapq.heappush(heap, (new_f, new_g, new_state))

                game.set_player_position(player_position)
                game.set_box_positions(boxes_position)
    return None, 0


## Solve

In [20]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar,
    "Weighted A*": solver_weighted_astar
}

In [21]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map, start_time)
    end_time = time.time()
    
    time_spended = end_time - start_time

    length_moves = 0
    if moves is not None:
        length_moves = len(moves)

    return moves, length_moves, method, time_spended, map, numof_visited_states            

In [22]:
import pandas as pd

def solve_all():
    maps = [f'map{map_num}' for map_num in range(min(map_file_indices), max(map_file_indices) + 1)]
    
    metrics = ['moves', 'length_moves', 'time_spent', 'numof_visited_states']
    columns = pd.MultiIndex.from_product([maps, metrics], names=['Map', 'Metric'])
    df = pd.DataFrame(index=list(SOLVERS.keys()), columns=columns)

    for map_num in range(min(map_file_indices), max(map_file_indices) + 1):
        map_name = f'map{map_num}'
        for method in SOLVERS.keys():
            moves, length_moves, _, time_spent, _, num_visited = solve(map_num, method)
            
            df.loc[method, (map_name, 'moves')] = moves
            df.loc[method, (map_name, 'length_moves')] = length_moves
            df.loc[method, (map_name, 'time_spent')] = time_spent
            df.loc[method, (map_name, 'numof_visited_states')] = num_visited

    return df
   

In [23]:
data = solve_all()

In [24]:
TIME_LIMIT = 60

print(solver_weighted_astar(load_map(9), time.time(), heuristic=heuristic_3, w=5))

('RURRRLLLDRDRULURURDRURDRDDLLUDRRULLRUULDDRDLDLULUUUUUUULULDLDRRURDDDDDLDR', 154035)


# Questions

<b> 1. </b> Defining the entire map as a state for each move would store a large amount of unnecessary information, especially in large maps with many cells. Since the locations of walls, portals, and empty spaces remain constant throughout the game, we can optimize storage by only saving the positions of the player and boxes. This significantly reduces memory usage and improves efficiency.

<b>2.</b> Actions are any operations that modify the state. Based on our state definition, each action represents the player's movement in one of four directions: Up (U), Right (R), Left (L), or Down (D).
- An action is valid if: 
    - There is no wall blocking the player's movement in the chosen direction
    - If a box is in the player's path, the next cell must be empty to allow the box to be pushed.

- An action can modify the state:
    - The player's position changes based on the selected direction.
    - If the player pushes a box, its position is updated accordingly.

also moving through the portals and pushing boxes are considered action.

<b>3.</b> Actions and States are defined in the previous parts. <b>Goal state</b> is an state in which all boxes are in their own goal location. <b>Initial State</b> is initial location of player and the boxes on the map.  

<b>4. </b> If we expand all possible moves at each steps it may be not optimal in all cases:

- Some paths lead to deadends for example a box can get stuck in a corner and can no longer reach its goal.
- Player can go forward and back without moving a box and this leads to a repeated state.
- moving in circles and not moving any box also can lead to repeated state.

We can use some strategies to avoid above issues:
- If a box get stuck in a corner, ignore that state and stop expanding it.
- Consider a visited set to avoid repeated states. 
- By defining a heuristic in A* algorithm, we first expand states that are more probably near to goal and avoid expanding states that are too far from the goal state.

To make space complexity optimal, we can just store location of player and boxes instead of the whole map(as we did in part 1).

<b>5.</b>

- BFS: First expands the states that have less height in tree of search.
    - Pros(+): Finds the optimal path in unweighted graphs
    - Cons(-): High Space complexity

- DFS: expands states in the depth first
    - Pros(+): Low Space complexity
    - Cons(-): answer is not optimal and it may be very long, agent could get stuck in loops and unnecessary iterations which leads unoptimal answers
        , can get stuck in infinite loops
- IDS: DFS in different levels of graph from 1 to end
    - Pros(+): less space complexity than BFS, better answers than DFS     
    - Cons(-): repeating DFS in different levels, takes a lot of time

- A*: expands states with minimum f = g + h
    - Pros(+): Fastest algorithm if we have a good heuristic
    - Cons(-): calculating a good heuristic may need a lot of time, and also finding one is not always easy

- Weighted A*: expands states with minimum f = g + alpha * h
    - Pros(+): Finds the goal faster in comparison to A*
    - Cons(-): its not optimal in comparison to A*


<table>
    <tr>
        <th>Algorithm</th>
        <th>Completeness</th>
        <th>Optimality</th>
    </tr>
    <tr>
        <td><strong>BFS</strong></td>
        <td>Yes</td>
        <td>Yes (if all moves have equal cost)</td>
    </tr>
    <tr>
        <td><strong>DFS</strong></td>
        <td>Yes (if depth is limited)</td>
        <td>No</td>
    </tr>
    <tr>
        <td><strong>IDS</strong></td>
        <td>Yes</td>
        <td>Yes (if all moves have equal cost)</td>
    </tr>
    <tr>
        <td><strong>A*</strong></td>
        <td>Yes</td>
        <td>Yes (with an admissible heuristic)</td>
    </tr>
    <tr>
        <td><strong>Weighted A*</strong></td>
        <td>Yes (but may not find the shortest path)</td>
        <td>No (approximates optimality)</td>
    </tr>
</table>


a. as discussed above, DFS doesnt find the optimal answer. but in some problems, optimality isnt important compared to space complexity. as long as space complexity of DFS is O(bd) and its very low, it way be used in some cases.

b. IDS provides an algorithm with space complexity near to DFS and optimality near to BFS. this algorithm solves the space problem for BFS and optimality issue for DFS.

<b>6.</b> Algorithms are implemented above. results are as expected, near to what we knew in theory.

<b>7. </b>

1. First heuristic is SUM of Manhatann distances of all boxes to their goal cells. The Manhatann distance of each box is minimum steps needed to move the box from its current postion to its goal cell. Moving a box decreases the heuristic at most by 1 (the reduction in Manhattan distance).
The cost of moving a box is always larger or equal to the decrease in heuristic.
But due to the portals, distance of a box to its goal could be less than the manhatann distance. So this is not addmisible and consistent. 

2. Second heuristic is SUM of Manhatann distances of boxes to their goal states and minimum distance of player to a box(which is not in goal cell). It includes the same Manhattan distance heuristic as First heuristic (which we confirmed is admissible). It also adds the Manhattan distance from the player to the closest box, which is an additional cost that will be required anyway.
The added term to heuristic changes by at most 1 when the player moves.
Since every move has a cost of at least 1, the heuristic does not increase more than the actual move cost.
But due to the portals, distance of a box to its goal could be less than the manhatann distance. So this is not addmisible and consistent. 

3. Third heuristic in addition to last two, considers a penalty for boxes that are in the corner of walls. like two previous heuristics, this is not admissible or consistent too.

4. Fourth heuristic is count of all boxes that are not in their goal places. this heuristic is obviously admissible and consistent because each box that is not on its goal, needs at leat one step to be moved to the goal. 


<b>8.</b>

In [25]:
map = load_map(6)
print('heuristic 1', solver_astar(map, start_time=time.time(), heuristic=heuristic_1))
print('heuristic 2', solver_astar(map, start_time=time.time(), heuristic=heuristic_2))
print('heuristic 3',solver_astar(map, start_time=time.time(), heuristic=heuristic_3))
print('heuristic 4',solver_astar(map, start_time=time.time(), heuristic=heuristic_4))
print()

map = load_map(7)
print('heuristic 1', solver_astar(map, start_time=time.time(), heuristic=heuristic_1))
print('heuristic 2', solver_astar(map, start_time=time.time(), heuristic=heuristic_2))
print('heuristic 3',solver_astar(map, start_time=time.time(), heuristic=heuristic_3))
print('heuristic 4',solver_astar(map, start_time=time.time(), heuristic=heuristic_4))
print()

map = load_map(8)
print('heuristic 1', solver_astar(map, start_time=time.time(), heuristic=heuristic_1))
print('heuristic 2', solver_astar(map, start_time=time.time(), heuristic=heuristic_2))
print('heuristic 3',solver_astar(map, start_time=time.time(), heuristic=heuristic_3))
print('heuristic 4',solver_astar(map, start_time=time.time(), heuristic=heuristic_4))

heuristic 1 ('DDDDDRRRLLLLLLLRUUUUUUUUUULRRRRRRR', 7036)
heuristic 2 ('DDDDDRRRLLLLLLLRUUUUUUUUUULRRRRRRR', 4324)
heuristic 3 ('DDDDDRRRLLLLLLLRUUUUUUUUUULRRRRRRR', 558)
heuristic 4 ('DDDDDRRRLLLLLLLRUUUUUUUUUULRRRRRRR', 12034)

heuristic 1 ('RURRDDDDLDRUUUULLLRDRDRDDLLDLLURLU', 47581)
heuristic 2 ('RURRDDDDLDRUUUULLLRDRDRDDLLDLLUUDR', 39430)
heuristic 3 ('RURRDDDDLDRLLDLLUUDRRURRUUULLLRDRDRDDL', 5595)
heuristic 4 ('RURRDDDDLDRUUUULLLRDRDRDDLLDLLURLU', 177473)

heuristic 1 ('URRRRRRRRRRRRRURDDR', 951)
heuristic 2 ('URRRRRRRRRRRRRURDDR', 105)
heuristic 3 ('URRRRRRRRRRRRRURDDR', 102)
heuristic 4 ('UURDLDRRDRURDR', 5050)


We know that A* gives the optimal solution if its heuristic is admissible. Fourth heuristic gives optimal solution cause its admissible. but three others are much faster and expand much less state as shown above. So the trade off is between speed and optimality. 
as long as there is not much difference in optimality of these heuristics (on avg), I would rather use the fastest on which is the third one.  

<b>9. </b>

In [26]:
for index in range(1, 11):
    print('Map', index, ':')
    display(data['map'+str(index)])

Map 1 :


Metric,moves,length_moves,time_spent,numof_visited_states
BFS,UDDULRR,7,0.0,40
DFS,RLLRDUU,7,0.001,8
IDS,UDDULRR,7,0.003001,189
A*,UDLRRLD,7,0.000999,14
Weighted A*,UDLRRLD,7,0.0,8


Map 2 :


Metric,moves,length_moves,time_spent,numof_visited_states
BFS,LUDDUL,6,0.0,18
DFS,LLRDUU,6,0.0,7
IDS,LUDDUL,6,0.001001,74
A*,LUDLRD,6,0.0,10
Weighted A*,LUDLRD,6,0.0,7


Map 3 :


...

Metric,moves,length_moves,time_spent,numof_visited_states
BFS,UURDLDRRDRURDR,14,0.073125,5083
DFS,RRRRLLLLDRRRRRRURRRRRRRRRRDLLLLLLLLLLLLLLLLDRR...,481,0.117722,8058
IDS,UURDLDRRDRURDR,14,0.141137,27473
A*,URRRRRRRRRRRRRURDDR,19,0.002275,102
Weighted A*,URRRRRRRRRRRRRURDDR,19,0.0,20


Map 9 :


Metric,moves,length_moves,time_spent,numof_visited_states
BFS,None,0,64.222582,-1
DFS,None,0,62.068868,-1
IDS,None,0,60.234966,-1
A*,None,0,60.853664,-1
Weighted A*,RURRRLLLDRDRULURURDRURDRDDLLUDRRULLRUULDDRDLDL...,73,5.85547,154035


Map 10 :


Metric,moves,length_moves,time_spent,numof_visited_states
BFS,RRRRRDRULURULLLULDRUUULDRDLDRRDRULURURDDRDLLLL,46,1.903981,228003
DFS,RRRRRLLLLLLLDRRRRRRRRRULRDLLLLLLLLLURURRRRRRRD...,998,1.723593,208951
IDS,RRRRRDRULURULLLULDRUUULDRDLDRRDRULURURDDRDLLLL,46,25.1071,2604381
A*,RRRRRDRULURULLLULDRUUULDRDLDRRDRULURURDDRDLLLL,46,1.342804,38834
Weighted A*,RRRRDRURULLLUURDLDRDDLLURRDRULURURDDRDLLLDLURU...,74,0.234308,6332


<b>10.</b> Comparing different weights for weighted A*

In [27]:
map = load_map(6)
print('alpha = 2 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 2))
print('alpha = 5 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 5))
print('alpha = 7 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 7))
print('alpha = 10 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 10))
print()

map = load_map(7)
print('alpha = 2 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 2))
print('alpha = 5 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 5))
print('alpha = 7 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 7))
print('alpha = 10 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 10))
print()

map = load_map(8)
print('alpha = 2 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 2))
print('alpha = 5 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 5))
print('alpha = 7 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 7))
print('alpha = 10 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 10))
print()

map = load_map(9)
print('alpha = 2 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 2))
print('alpha = 5 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 5))
print('alpha = 7 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 7))
print('alpha = 10 ->', solver_weighted_astar(map, start_time=time.time(), heuristic=heuristic_3, w = 10))
print()



alpha = 2 -> ('UUUUURRRLLLLLLLDDDDDDDDDRDLRRRRRRR', 50)
alpha = 5 -> ('UUUUURRRLLLLLLLDDDDDDDDDRDLRRRRRRR', 48)
alpha = 7 -> ('UUUUURRRLLLLLLLDDDDDDDDDRDLRRRRRRR', 52)
alpha = 10 -> ('UUUUURRRLLLLLLLDDDDDDDDDRDLRRRRRRR', 56)

alpha = 2 -> ('RURRDLLLRRRDDDLDRUUUULLDRDRDDLLDLLUUDR', 1852)
alpha = 5 -> ('RURRDDDDLDRUUUUULLDDRDULUURRDLLLRDRDRDDLLDLLUUDR', 870)
alpha = 7 -> ('RURRDDDDLDRUUUUULLDDRDULUURRDLLLRDRDRDDLLDLLUUDR', 679)
alpha = 10 -> ('RURRDDDDLDRUUUUULLDDRDULUURRDLLLRDRDRDDLLDLLUUDR', 710)

alpha = 2 -> ('URRRRRRRRRRRRRURDDR', 20)
alpha = 5 -> ('URRRRRRRRRRRRRURDDR', 20)
alpha = 7 -> ('URRRRRRRRRRRRRURDDR', 20)
alpha = 10 -> ('URRRRRRRRRRRRRURDDR', 20)

alpha = 2 -> ('RURRRLDDLULURRURDRDRDLDLULUUUUUULLLURRURDDDDDDLDRURRURDRDLL', 422541)
alpha = 5 -> ('RURRRLLLDRDRULURURDRURDRDDLLUDRRULLRUULDDRDLDLULUUUUUUULULDLDRRURDDDDDLDR', 154035)
alpha = 7 -> ('RURRRLLLDRDRULURURDRURDRDDLLUDRRULLRUULDDRDLDLULUUUUUUULULDLDRRURDDDDDLDR', 475791)
alpha = 10 -> (None, -1)



As seen above, for weight w = 5, less states are expanded that means it is faster. and also in for map nine, only responding weight is 5 and others exceede the time limit. 